# Demostración Curvas Elípticas

Primero, escogemos la curva Curve25519 recomendada en el RFC7748 (https://tools.ietf.org/html/rfc7748#page-4).
Damos el cuerpo, la curva y el punto G base del subgrupo (también recomendado en el artículo).

In [12]:
K = GF(2^(255)-19); # Cuerpo
E =  EllipticCurve(K, [0, 486662, 0, 1, 0]); # Curva
G = E(9,14781619447589544791020593568409986887264606134616475288964881837755586237401); # Generador del subgrupo
n = 2^252 + 0x14def9dea2f79cd65812631a5cf5d3ed; # Orden subgrupo generado por G

Esta el la función para generación de la llave privada (número) y la pública (punto)

In [13]:
def obtenerClaves(G, n):
    d = ZZ.random_element(1, n-1)
    H = d*G
    
    return d,H #Privada / Pública 

Obtenemos una pareja de claves

In [31]:
dA, HA = obtenerClaves(G, n); dA, HA

(6466217458795460239168546665254127222386948803784032710039152625253381618434,
 (27501174150858854412187924747415082264060602229435582116045824313236092380309 : 40115898585552385400975026573704901568990540539599444469911269012302469956777 : 1))

## ECDSA

Función para firmar un mensaje y otra para verificar la firma

In [27]:
def firmarEC(mensaje, G, n, d): #Menasaje / Generador subgrupo / Orden subgrupo / Clave privada
    z = hash(mensaje) % n; # Hash del mensaje
    
    k = ZZ.random_element(1, n-1)
    P = k*G; 
    r = P[0]; 
    r = Integer(r) % n;
    k_inv = inverse_mod(k,n);
    s = k_inv*(z+r*d) % n;

    return r, s

def verificarFirmaEC(mensaje, G, n, H, r, s): #Mensaje recibido / Generador subgrupo / Orden subgrupo / clave pública / valores firma (r, s)
    z = hash(mensaje) % n; # Hash del mensaje
    
    u1 = ((1/s)*z) % n; 
    u2 = ((1/s)*r) % n; 
    P1 = u1*G + u2*H; 
    xP = Integer(P1[0]) % n; 
    
    igual = False
    if r == xP:
        igual = True
    
    return igual

In [34]:
mensaje1 = "Demostración de la curva elíptica"
mensaje2 = "Demostración de la curva elípticaa"

In [39]:
r, s = firmarEC(mensaje1, G, n, dA)

In [40]:
verificarFirmaEC(mensaje1, G, n, HA, r, s)

True

In [41]:
verificarFirmaEC(mensaje2, G, n, HA, r, s)

False